DBInterface
==========

.. warning:: This class is in development and has not been released.

The [DBInterface](api/kineticstoolkit.dbinterface.rst) class is used exclusively at the [Mobility and Adaptive Sports Research Lab](https://felixchenier.uqam.ca) and may be removed from ktk someday. Its aim is to interface with the BIOMEC database (https://felixchenier.uqam.ca/biomec) to fetch all non-personal information about a specified ongoing project.

It is necessary to understand how data files are stored in BIOMEC before reading this tutorial.

Please note that the user/password combination used in this tutorial is not valid, and that you should get access to BIOMEC to use ktk.dbinterface.

In [ ]:
import kineticstoolkit.lab as ktk
import os
import shutil

Connecting to a BIOMEC project
---------------------------------------
The class constructor connects to the project and asks the user's credentials
and the folder where the data files are stored.

    project = ktk.DBInterface(project_label)

For example:

    project = ktk.DBInterface('FC_XX18A')

The constructor can also be run non-interactively:

In [ ]:
project_label = 'dummyProject'
user = 'dummyUser'
password = 'dummyPassword'
root_folder = 'data/dbinterface/FC_XX18A'
url = ''
url = 'http://localhost/biomec'  # This line is only for this tutorial,
                                 # please don't execute it.

project = ktk.DBInterface(project_label,
                          user=user,
                          password=password,
                          root_folder=root_folder,
                          url=url)

Navigating in the project
-------------------------
Just typing ``project`` gives an overview of the project's content.

In [ ]:
project

The method ``get`` is used to extract the project's contents. It always returns
a dict with the fields corresponding to the request. For example:

In [ ]:
project.get()

In [ ]:
project.get('P1')

In [ ]:
project.get('P1')['Sessions']

In [ ]:
project.get('P1', 'GymnaseN1')

In [ ]:
project.get('P1', 'GymnaseN1')['Trials']

In [ ]:
project.get('P1', 'GymnaseN1', 'Run1')

In [ ]:
project.get('P1', 'GymnaseN1', 'Run1')['Files']

In [ ]:
project.get('P1', 'GymnaseN1', 'Run1', 'Kinematics')

In [ ]:
project.get('P1', 'GymnaseN1', 'Run1', 'Kinematics')['FileName']

Saving data to a BIOMEC referenced file
---------------------------------------
The ktk library provides the function ktk.save to save a variable to a JSON file. The ktk.save function is helpful to save temporary results.

However, sometimes we need to save results to BIOMEC so that these results become new inputs for subsequent work. In these case, we use the ktk.DBInterface's ``save`` method.

For example, let's say we just synchronized the kinematics for Run1 of participant 1:

In [ ]:
synced_kinematics = {'dummy_data':
                     'Normally we would save something more useful'}

We can save these kinematics as a file that is referenced in BIOMEC, using:

In [ ]:
project.save('P1', 'GymnaseN1', 'Run1', 'SyncedKinematics', synced_kinematics)

This creates the file entry in BIOMEC if needed, then save the file with
a relevant name into the project folder.

We can now obtain the file name using the ``get`` method introduced precedently.

In [ ]:
project.get('P1', 'GymnaseN1', 'Run1', 'SyncedKinematics')['FileName']

Loading data from a BIOMEC referenced file
------------------------------------------
To load back data saved to BIOMEC, we use the ktk.DBInterface's ``load``
method.

In [ ]:
test = project.load('P1', 'GymnaseN1', 'Run1', 'SyncedKinematics')

test

Let's do a little clean up before going on.

In [ ]:
shutil.rmtree(root_folder + '/SyncedKinematics')

Dealing with external software
------------------------------
The DBInterface's ``save`` and ``load`` methods work very well for data that were processed in Python using ktk. However, things may get complicated when using external software to process data.

In this example, we will synchronize the kinematics using an external synchronizing tool, then enter the resulting files into BIOMEC. We will work with these files:

In [ ]:
file_list = []
for trial in ['Walk1', 'Walk2', 'Run1', 'Run2']:
    file_list.append(project.get(
        'P1', 'GymnaseN1', trial, 'Kinematics')['FileName'])

file_list

Let say we synchronized these files using an external software, and then we
exported the synchronized files into a separate folder.

(Here we will simply copy those files into a separate folder).

In [ ]:
os.mkdir(root_folder + '/synchronized_files')

for file in file_list:
    dest_file = file.replace(root_folder, root_folder + '/synchronized_files')
    shutil.copyfile(file, dest_file)

os.listdir(root_folder + '/synchronized_files')

All is good, but the dbfids in the new ``synchronized_files`` folder refer to
the original ``Kinematics`` file type, not to the ``SyncedKinematics`` file
type. Moreover, there are now duplicate dbfids in the project:

In [ ]:
project.refresh()

In [ ]:
project.duplicates

Therefore we need to assign new dbfids to the files we just synchronized, so that they refer to ``SyncedKinematics`` entries in BIOMEC. The method ``batch_fix_file_type`` will help.

In [ ]:
project.batch_fix_file_type(root_folder + '/synchronized_files',
                            'SyncedKinematics',
                            create_file_entries=True,
                            dry_run=False)

Now let see what happened in the ``synchronized_files`` folder:

In [ ]:
os.listdir(root_folder + '/synchronized_files')

The files' dbfid have been updated so they now refer to ``SyncedKinematics`` and not to ``Kinematics`` anymore. Moreover, the project does not have duplicate dbfids anymore:

In [ ]:
project.refresh()

Let's do a little clean up before going on.

In [ ]:
shutil.rmtree(root_folder + '/synchronized_files')
project.refresh()

Including information in file names
-----------------------------------
It can be difficult to deal with a bunch of numbered files without knowing their signification without looking in BIOMEC. The ``tag_files`` method allows adding the trial name to the file names, so that their context is a bit clearer and less error-prone.

In [ ]:
os.listdir(root_folder)

Include the trial name in the file names:

In [ ]:
project.tag_files(include_trial_name=True, dry_run=False)

os.listdir(root_folder)

Remove the trial name from the file names:

In [ ]:
project.tag_files(include_trial_name=False, dry_run=False)

os.listdir(root_folder)

For more information on DBInterface, please check the [API Reference](api/kineticstoolkit.dbinterface.rst).